In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2024-07-21 02:49:19.512995: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-21 02:49:19.716906: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 02:49:19.716928: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 02:49:19.718012: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-21 02:49:19.804790: I tensorflow/core/platform/cpu_feature_g

2.14.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-07-21 02:49:22.005525: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-21 02:49:22.022717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-21 02:49:22.022750: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


# Onset 모델 학습

In [ ]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, TimeDistributed, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.mixed_precision import Policy, set_global_policy

# 혼합 정밀도 정책 설정
policy = Policy('mixed_float16')
set_global_policy(policy)

def data_generator(cqt_paths, stft_paths, onset_paths, model):
    for cqt_path, stft_path, onset_path in zip(cqt_paths, stft_paths, onset_paths):
        CQT = np.load(cqt_path)
        STFT = np.load(stft_path)
        ONSET = np.load(onset_path)

        # 정규화
        CQT = CQT / np.max(CQT)
        STFT = STFT / np.max(STFT)

        model.reset_states()  # 한 곡마다 모델 상태 초기화
        for cqt, stft, onset in zip(CQT, STFT, ONSET):
            yield ({'cqt_input': cqt, 'stft_input': stft}, onset)  # 한 세그먼트씩 반환

def build_model():
    cqt_input = Input(batch_input_shape=(10, 100, 264), name='cqt_input')
    stft_input = Input(batch_input_shape=(10, 100, 257), name='stft_input')

    cqt_lstm = Bidirectional(LSTM(128, activation='tanh', return_sequences=True, stateful=True))(cqt_input)
    stft_lstm = Bidirectional(LSTM(128, activation='tanh', return_sequences=True, stateful=True))(stft_input)

    concatenated = Concatenate()([cqt_lstm, stft_lstm])

    onset_out = TimeDistributed(Dense(88, activation='sigmoid', kernel_initializer='he_normal', name='onset_output'))(concatenated)
    model = Model(inputs=[cqt_input, stft_input], outputs=onset_out)

    return model

def train(train_cqt, train_stft, train_onset, valid_cqt, valid_stft, valid_onset):
    train_cqt_paths, train_stft_paths, train_onset_paths = [], [], []
    valid_cqt_paths, valid_stft_paths, valid_onset_paths = [], [], []

    for cqt_path, stft_path, onset_path in zip(glob.glob(train_cqt), glob.glob(train_stft), glob.glob(train_onset)):
        train_cqt_paths.append(cqt_path)
        train_stft_paths.append(stft_path)
        train_onset_paths.append(onset_path)

    for cqt_path, stft_path, onset_path in zip(glob.glob(valid_cqt), glob.glob(valid_stft), glob.glob(valid_onset)):
        valid_cqt_paths.append(cqt_path)
        valid_stft_paths.append(stft_path)
        valid_onset_paths.append(onset_path)

    output_signature = (
        {
            'cqt_input': tf.TensorSpec(shape=(100, 264), dtype=tf.float32),
            'stft_input': tf.TensorSpec(shape=(100, 257), dtype=tf.float32)
        },
        tf.TensorSpec(shape=(100, 88), dtype=tf.int8)
    )

    model = build_model()

    train_set = tf.data.Dataset.from_generator(lambda: data_generator(train_cqt_paths, train_stft_paths, train_onset_paths, model), output_signature=output_signature)
    valid_set = tf.data.Dataset.from_generator(lambda: data_generator(valid_cqt_paths, valid_stft_paths, valid_onset_paths, model), output_signature=output_signature)
    train_set = train_set.batch(10, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    valid_set = valid_set.batch(10, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

    checkpoint = ModelCheckpoint('models/onset_detector_v2.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    early_stop = EarlyStopping(patience=5, monitor='val_loss', verbose=1, mode='auto')

    optimizer = Adam(learning_rate=0.0005)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.fit(train_set, validation_data=valid_set, epochs=2, shuffle=False, callbacks=[checkpoint, early_stop])
    model.save('models/onset_last_v2.h5')

if __name__ == '__main__':
    train(
        'data/preprocessed/trainX/cqt/*.npy',
        'data/preprocessed/trainX/stft/*.npy',
        'data/preprocessed/trainONSET/*.npy',
        'data/preprocessed/validX/cqt/*.npy',
        'data/preprocessed/validX/stft/*.npy',
        'data/preprocessed/validONSET/*.npy')


# Offset 모델 학습

In [ ]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, TimeDistributed, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.mixed_precision import Policy, set_global_policy

# 혼합 정밀도 정책 설정
policy = Policy('mixed_float16')
set_global_policy(policy)

def data_generator(cqt_paths, stft_paths, offset_paths, model):
    for cqt_path, stft_path, offset_path in zip(cqt_paths, stft_paths, offset_paths):
        CQT = np.load(cqt_path)
        STFT = np.load(stft_path)
        OFFSET = np.load(offset_path)

        # 정규화
        CQT = CQT / np.max(CQT)
        STFT = STFT / np.max(STFT)

        model.reset_states()  # 한 곡마다 모델 상태 초기화
        for cqt, stft, offset in zip(CQT, STFT, OFFSET):
            yield ({'cqt_input': cqt, 'stft_input': stft}, offset)  # 한 세그먼트씩 반환

def build_model():
    cqt_input = Input(batch_input_shape=(10, 100, 264), name='cqt_input')
    stft_input = Input(batch_input_shape=(10, 100, 257), name='stft_input')

    cqt_lstm = Bidirectional(LSTM(128, activation='tanh', return_sequences=True, stateful=True))(cqt_input)
    stft_lstm = Bidirectional(LSTM(128, activation='tanh', return_sequences=True, stateful=True))(stft_input)

    concatenated = Concatenate()([cqt_lstm, stft_lstm])

    offset_out = TimeDistributed(Dense(88, activation='sigmoid', kernel_initializer='he_normal', name='offset_output'))(concatenated)
    model = Model(inputs=[cqt_input, stft_input], outputs=offset_out)

    return model

def train(train_cqt, train_stft, train_offset, valid_cqt, valid_stft, valid_offset):
    train_cqt_paths, train_stft_paths, train_offset_paths = [], [], []
    valid_cqt_paths, valid_stft_paths, valid_offset_paths = [], [], []

    for cqt_path, stft_path, offset_path in zip(glob.glob(train_cqt), glob.glob(train_stft), glob.glob(train_offset)):
        train_cqt_paths.append(cqt_path)
        train_stft_paths.append(stft_path)
        train_offset_paths.append(offset_path)

    for cqt_path, stft_path, offset_path in zip(glob.glob(valid_cqt), glob.glob(valid_stft), glob.glob(valid_offset)):
        valid_cqt_paths.append(cqt_path)
        valid_stft_paths.append(stft_path)
        valid_offset_paths.append(offset_path)

    output_signature = (
        {
            'cqt_input': tf.TensorSpec(shape=(100, 264), dtype=tf.float32),
            'stft_input': tf.TensorSpec(shape=(100, 257), dtype=tf.float32)
        },
        tf.TensorSpec(shape=(100, 88), dtype=tf.int8)
    )

    model = build_model()

    train_set = tf.data.Dataset.from_generator(lambda: data_generator(train_cqt_paths, train_stft_paths, train_offset_paths, model), output_signature=output_signature)
    valid_set = tf.data.Dataset.from_generator(lambda: data_generator(valid_cqt_paths, valid_stft_paths, valid_offset_paths, model), output_signature=output_signature)
    train_set = train_set.batch(10, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    valid_set = valid_set.batch(10, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

    checkpoint = ModelCheckpoint('models/offset_detector_v2.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    early_stop = EarlyStopping(patience=5, monitor='val_loss', verbose=1, mode='auto')

    optimizer = Adam(learning_rate=0.0005)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.fit(train_set, validation_data=valid_set, epochs=2, shuffle=False, callbacks=[checkpoint, early_stop])
    model.save('models/offset_last_v2.h5')

if __name__ == '__main__':
    train(
        'data/preprocessed/trainX/cqt/*.npy',
        'data/preprocessed/trainX/stft/*.npy',
        'data/preprocessed/trainOFFSET/*.npy',
        'data/preprocessed/validX/cqt/*.npy',
        'data/preprocessed/validX/stft/*.npy',
        'data/preprocessed/validOFFSET/*.npy')
